In [1]:
import tensorflow as tf
import tensorflow_probability as tfp

c:\users\sanjay\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
c:\users\sanjay\anaconda3\lib\site-packages\tensorflow\python\util\tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
c:\users\sanjay\anaconda3\lib\site-packages\tensorflow\python\util\tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
c:\users\sanjay\anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  ret

Instructions for updating:
Use the retry module or similar alternatives.


c:\users\sanjay\anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
c:\users\sanjay\anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
c:\users\sanjay\anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
c:\users\sanjay\anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
c:\users\sanjay\anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
c:\users\sanjay\anaconda3\lib\importlib\_bootstrap

In [2]:
import numpy as np
time = np.linspace(0,10,100)
sinus = np.sin(time)
noise1 = np.random.normal(0,0.1,time.shape)
noise2 = np.random.normal(0,0.1,time.shape)
noised_sinus1 = sinus + noise1
noised_sinus2 = sinus + noise2
noised_sinus = np.hstack((noised_sinus1,noised_sinus2))
time_combined = np.hstack((time,time))
noised_sinus = noised_sinus.reshape(-1,1)
time_combined = time_combined.reshape(-1,1)


In [3]:
print(time_combined.shape)

(200, 1)


In [4]:
%matplotlib ipympl
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.plot(time, noised_sinus1)
ax.plot(time, noised_sinus2)

c:\users\sanjay\anaconda3\lib\site-packages\ipympl\backend_nbagg.py:125: DeprecationWarning: metadata {'sync': True} was set from the constructor. With traitlets 4.1, metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
  _model_module = Unicode('jupyter-matplotlib', sync=True)
c:\users\sanjay\anaconda3\lib\site-packages\ipywidgets\widgets\widget.py:409: DeprecationWarning: Passing unrecoginized arguments to super(FigureCanvasNbAgg).__init__().
__init__() missing 1 required positional argument: 'figure'
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)


FigureCanvasNbAgg()

In [35]:
tf.reset_default_graph()
time_nn = tf.placeholder(tf.float32, shape=(None, 1)) 

model = tf.keras.Sequential([
         tfp.layers.DenseFlipout(10, activation=tf.nn.tanh),
         tfp.layers.DenseFlipout(10, activation=tf.nn.tanh),
         tfp.layers.DenseFlipout(1, activation=tf.nn.tanh),
  ])
predictions = model(time_nn)

labels = tf.placeholder(tf.float32, shape=(None, 1))  

loss = tf.losses.mean_squared_error(labels, predictions)            # define loss
lossSummary = tf.summary.scalar('Loss', loss)
sgd_optimizer = tfp.optimizer.StochasticGradientLangevinDynamics(0.01,preconditioner_decay_rate=0.95)
#sgd_optimizer = tf.train.AdamOptimizer()
var = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
gradients = tf.gradients(xs=var, ys=loss)
sgd_op = sgd_optimizer.apply_gradients(zip(gradients, var))

c:\users\sanjay\anaconda3\lib\site-packages\tensorflow\python\util\tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
c:\users\sanjay\anaconda3\lib\site-packages\tensorflow\python\util\tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
c:\users\sanjay\anaconda3\lib\site-packages\tensorflow\python\util\tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
c:\users\sanjay\anaconda3\lib\site-packages\tensorflow\python\util\tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec

NotImplementedError: 

In [32]:
import time as Time
sess=tf.Session()  
sess.run(tf.global_variables_initializer())
train_writer = tf.summary.FileWriter('summary\\'+Time.strftime('%H_%M_%S'), sess.graph)
for i in range(20000):
    feed_dict={time_nn:time_combined, labels:noised_sinus}
    _, lossS = sess.run([sgd_op, lossSummary], feed_dict=feed_dict) 
    train_writer.add_summary(lossS, i)

In [33]:
feed_dict={time_nn:time.reshape(-1,1)}
PredVec = sess.run([predictions], feed_dict=feed_dict) 
%matplotlib ipympl
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.plot(time, noised_sinus1)
ax.plot(time, noised_sinus2)
ax.plot(time, PredVec[0])

c:\users\sanjay\anaconda3\lib\site-packages\ipywidgets\widgets\widget.py:409: DeprecationWarning: Passing unrecoginized arguments to super(FigureCanvasNbAgg).__init__().
__init__() missing 1 required positional argument: 'figure'
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)


FigureCanvasNbAgg()